In [2]:
import numpy as np
import pandas as pd
import xlsxwriter

In [3]:
files = ["confirmed_global.csv"]#, "deaths_global.csv", "recovered_global.csv"]

writer = pd.ExcelWriter('covid19.xlsx', engine = 'xlsxwriter')
workbook = writer.book

for f in files:
    df = pd.read_csv("https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_" + f)
    df = pd.melt(df, id_vars=df.columns[:4], value_vars=df.columns[4:], var_name="date", value_name="count")
    
    #萃取年分資料，刪除地區欄位
    df['year'] =  '20' + df['date'].str.split('/').str.get(2)
    del df['Province/State']
    print(df)
    
    #提取台/美/日人數總和為df1
    df1 = df[df['Country/Region'].isin(['Taiwan*', 'US', 'Japan'])]
    df1 = df1[['Country/Region', 'year', 'count']]
    df1 = df1.groupby(['Country/Region', 'year']).max()
    df1.reset_index(inplace=True)
    df1 = np.array(df1).tolist()
    print(df1)
    
    #將df寫成excel工作表，再插入df1的資料
    df.to_excel(writer, f, index=False, header=False, startrow=1, startcol=0)
    worksheet = writer.sheets[f]
    format_title = workbook.add_format({'valign':'vcenter', 'align':'center', 'bold':True, 'bg_color':'cccccc', 'font_size':14})
    
    worksheet.write_row('A1', ['國家', '緯度', '經度', '日期', '人數', '年'], cell_format=format_title) #插入df標題
    worksheet.write_row('H3', ['國家','年份', '總人數'], cell_format=format_title) #插入df1標題
    worksheet.set_column('J:J', 20)
    worksheet.write_row('H4', df1[0])
    worksheet.write_row('H5', df1[1])
    worksheet.write_row('H6', df1[2])
    worksheet.write_row('H7', df1[3])
    worksheet.write_row('H8', df1[4])
    worksheet.write_row('H9', df1[5])
    worksheet.write_row('H10', df1[6])
    worksheet.write_row('H11', df1[7])
    worksheet.write_row('H12', df1[8])
    
    #長條圖
    chart = workbook.add_chart({'type':'column'})
    chart.add_series({
        'categories':f'={f}!$I$4:$I$6',
        'values':f'={f}!$J$4:$J$6',
        'name':f'={f}!$H4'
    })
    chart.add_series({
        'categories':f'={f}!$I$7:$I$9',
        'values':f'={f}!$J$7:$J$9',
        'name':f'={f}!$H$7'
    })
    chart.add_series({
        'categories':f'={f}!$I$10:$I$12',
        'values':f'={f}!$J$10:$J$12',
        'name':f'={f}!$H$10'
    })
    worksheet.insert_chart('L3', chart)
    
writer.save()
workbook.close()

              Country/Region        Lat        Long     date   count  year
0                Afghanistan  33.939110   67.709953  1/22/20       0  2020
1                    Albania  41.153300   20.168300  1/22/20       0  2020
2                    Algeria  28.033900    1.659600  1/22/20       0  2020
3                    Andorra  42.506300    1.521800  1/22/20       0  2020
4                     Angola -11.202700   17.873900  1/22/20       0  2020
...                      ...        ...         ...      ...     ...   ...
219811    West Bank and Gaza  31.952200   35.233200   3/5/22  649971  2022
219812  Winter Olympics 2022  39.904200  116.407400   3/5/22     530  2022
219813                 Yemen  15.552727   48.516388   3/5/22   11775  2022
219814                Zambia -13.133897   27.849332   3/5/22  313744  2022
219815              Zimbabwe -19.015438   29.154857   3/5/22  239019  2022

[219816 rows x 6 columns]
[['Japan', '2020', 235749], ['Japan', '2021', 1732296], ['Japan', '2022',